In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('../data/02_GameSpy_Scraping_MergeAllDataSet.csv', index_col=0)

In [25]:
#Reformattage et enrichissement de la date de sortie 
df['Year2'] = df['DateSortie'].apply(lambda x: x[:4])
df['Year2'] = df['Year2'].astype(float)
df['Year'] = df['Year'].fillna(df['Year2'])
df
df = df.drop(['Year2', 'DateSortie'], axis=1)
df['Year'] = df['Year'].replace(0, np.nan)
df = df.dropna(axis = 0, how = 'all', subset = ['Year'])
df['Year'] = df['Year'].astype(int).astype(str)
df['Mois'] = df['Mois'].astype(int).astype(str)
for i,row in df.iterrows():
    if len(df.loc[i, 'Mois']) == 1:
        df.loc[i, 'Date_Sortie'] = df.loc[i,'Year']+'-0'+df.loc[i,'Mois']+'-01'
    else:
        df.loc[i, 'Date_Sortie'] = df.loc[i,'Year']+'-'+df.loc[i,'Mois']+'-01'
df['Date_Sortie'] = pd.to_datetime(df['Date_Sortie'])

# ROLLING MEANS PUBLISHERS
Création des moyennes mobiles des ventes, des scores, des rates et des reviews

In [26]:
df_publisher = df.sort_values(by = ['Publisher','Date_Sortie'], ascending=True)
publishers = df_publisher['Publisher'].unique()
publishers = np.delete(publishers, -1)
df_publisher['RM_Publisher'] = np.nan
df_publisher["RM_Publisher_score"] = np.nan
df_publisher["RM_Publisher_rate"] = np.nan
df_publisher["RM_Publisher_reviews"] = np.nan
df_rm1on3 = pd.DataFrame(columns=df_publisher.columns)
for publisher in publishers:
    df_temp = df_publisher[df_publisher['Publisher']==publisher]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
    elif nb_games == 2:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
    elif nb_games == 3:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
        df_temp.iloc[2,-4] = (df_temp.iloc[0:1,13]).mean()
        df_temp.iloc[2,-3] = (df_temp.iloc[0:1,6]).mean()
        df_temp.iloc[2,-2] = (df_temp.iloc[0:1,7]).mean()
        df_temp.iloc[2,-1] = (df_temp.iloc[0:1,8]).mean()
    else:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]        
        for i in range(2,nb_games):
            df_temp.iloc[i,-4] = (df_temp.iloc[(i-2):(i-1),13]).mean()
            df_temp.iloc[i,-3] = (df_temp.iloc[(i-2):(i-1),6]).mean()
            df_temp.iloc[i,-2] = (df_temp.iloc[(i-2):(i-1),7]).mean()
            df_temp.iloc[i,-1] = (df_temp.iloc[(i-2):(i-1),8]).mean()
    df_rm1on3 = pd.concat([df_rm1on3,df_temp], axis = 0)

In [27]:
#Merge avec le dataset importé
df_rm = df.merge(right=df_rm1on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie',], how='left')

# ROLLING MEANS STUDIOS
Création des moyennes mobiles des ventes, des scores, des rates et des reviews

In [28]:
df_studio = df_rm.sort_values(by = ['Studio','Date_Sortie'], ascending=True)
studios = df_studio['Studio'].unique()
studios = np.delete(studios, -1)
df_studio['RM_Studio'] = np.nan
df_studio["RM_Studio_score"] = np.nan
df_studio["RM_Studio_rate"] = np.nan
df_studio["RM_Studio_reviews"] = np.nan
df_rm2on3 = pd.DataFrame(columns=df_studio.columns)
for studio in studios:
    df_temp = df_studio[df_studio['Studio']==studio]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
    elif nb_games == 2:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
    elif nb_games == 3:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
        df_temp.iloc[2,-4] = (df_temp.iloc[0:1,13]).mean()
        df_temp.iloc[2,-3] = (df_temp.iloc[0:1,6]).mean()
        df_temp.iloc[2,-2] = (df_temp.iloc[0:1,7]).mean()
        df_temp.iloc[2,-1] = (df_temp.iloc[0:1,8]).mean()
    else:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]        
        for i in range(2,nb_games):
            df_temp.iloc[i,-4] = (df_temp.iloc[(i-2):(i-1),13]).mean()
            df_temp.iloc[i,-3] = (df_temp.iloc[(i-2):(i-1),6]).mean()
            df_temp.iloc[i,-2] = (df_temp.iloc[(i-2):(i-1),7]).mean()
            df_temp.iloc[i,-1] = (df_temp.iloc[(i-2):(i-1),8]).mean()
    df_rm2on3 = pd.concat([df_rm2on3,df_temp], axis = 0)

In [29]:
#Merge avec le dataset importé enrichi des rolling means publishers
df_rm = df_rm.merge(right=df_rm2on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie',
        'RM_Publisher', 'RM_Publisher_score', 'RM_Publisher_rate', 'RM_Publisher_reviews'], how='left')

In [30]:
#Gestion des NaN : remplacement des NaN Studio par les moyennes mobiles des publishers
df_rm["RM_Studio"]=df_rm["RM_Studio"].fillna(df_rm["RM_Publisher"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Publisher_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Publisher_rate"])
df_rm["RM_Studio_reviews"]=df_rm["RM_Studio_reviews"].fillna(df_rm["RM_Publisher_reviews"])

# ROLLING MEANS LICENCES
Création des moyennes mobiles des ventes, des scores, des rates et des reviews

In [31]:
df_licence = df_rm.sort_values(by = ['GK_licence','Date_Sortie'], ascending=True)
licences = df_licence['GK_licence'].unique()
licences = np.delete(licences, -1)
df_licence['RM_Licence'] = np.nan
df_licence["RM_Licence_score"] = np.nan
df_licence["RM_Licence_rate"] = np.nan
df_licence["RM_Licence_reviews"] = np.nan
df_rm3on3 = pd.DataFrame(columns=df_licence.columns)
for licence in licences:
    df_temp = df_licence[df_licence['GK_licence']==licence]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
    elif nb_games == 2:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
    elif nb_games == 3:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
        df_temp.iloc[2,-4] = (df_temp.iloc[0:1,13]).mean()
        df_temp.iloc[2,-3] = (df_temp.iloc[0:1,6]).mean()
        df_temp.iloc[2,-2] = (df_temp.iloc[0:1,7]).mean()
        df_temp.iloc[2,-1] = (df_temp.iloc[0:1,8]).mean()
    else:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]        
        for i in range(2,nb_games):
            df_temp.iloc[i,-4] = (df_temp.iloc[(i-2):(i-1),13]).mean()
            df_temp.iloc[i,-3] = (df_temp.iloc[(i-2):(i-1),6]).mean()
            df_temp.iloc[i,-2] = (df_temp.iloc[(i-2):(i-1),7]).mean()
            df_temp.iloc[i,-1] = (df_temp.iloc[(i-2):(i-1),8]).mean()
    df_rm3on3 = pd.concat([df_rm3on3,df_temp], axis = 0)

In [32]:
#Merge avec le dataset importé enrichi des rolling means publishers et studios
df_rm = df_rm.merge(right=df_rm3on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie',
        'RM_Publisher', 'RM_Publisher_score', 'RM_Publisher_rate', 'RM_Publisher_reviews',
        'RM_Studio', 'RM_Studio_score', 'RM_Studio_rate', 'RM_Studio_reviews'], how='left')

In [33]:
#Gestion des NaN : remplacement des NaN Licence par les moyennes mobiles des studios
df_rm["RM_Licence"]=df_rm["RM_Licence"].fillna(df_rm["RM_Studio"])
df_rm["RM_Licence_score"]=df_rm["RM_Licence_score"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Licence_rate"]=df_rm["RM_Licence_rate"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Licence_reviews"]=df_rm["RM_Licence_reviews"].fillna(df_rm["RM_Studio_reviews"])

In [34]:
#Gestion des NaN : remplacement des NaN Studio par les moyennes mobiles des Licences
df_rm["RM_Studio"]=df_rm["RM_Studio"].fillna(df_rm["RM_Licence"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Licence_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Licence_rate"])
df_rm["RM_Studio_reviews"]=df_rm["RM_Studio_reviews"].fillna(df_rm["RM_Licence_reviews"])

#Gestion des NaN : remplacement des NaN Publisher par les moyennes mobiles des Studios
df_rm["RM_Publisher"]=df_rm["RM_Publisher"].fillna(df_rm["RM_Studio"])
df_rm["RM_Publisher_score"]=df_rm["RM_Publisher_score"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Publisher_rate"]=df_rm["RM_Publisher_rate"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Publisher_reviews"]=df_rm["RM_Publisher_reviews"].fillna(df_rm["RM_Studio_reviews"])

In [35]:
#Gestions des NaN : remplacement des moyennes mobiles des scores par celles des rates
df_rm["RM_Publisher_score"]=df_rm["RM_Publisher_score"].fillna(df_rm["RM_Publisher_rate"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Licence_score"]=df_rm["RM_Licence_score"].fillna(df_rm["RM_Licence_rate"])

#Gestions des NaN : remplacement des moyennes mobiles des rates par celles des scores
df_rm["RM_Publisher_rate"]=df_rm["RM_Publisher_rate"].fillna(df_rm["RM_Publisher_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Licence_rate"]=df_rm["RM_Licence_rate"].fillna(df_rm["RM_Licence_score"])

In [36]:
#Gestions des NaN : remplacement des NaN des licences par les noms de jeux
df_rm["GK_licence"]=df_rm["GK_licence"].fillna(df_rm["Name"])
#Gestions des NaN : remplacement des NaN des studios par les licences
df_rm['Studio']=df_rm['Studio'].fillna(df_rm['GK_licence'])
#Gestions des NaN : remplacement des NaN des publishers par les studios
df_rm['Publisher']=df_rm['Publisher'].fillna(df_rm['Studio'])

#Ceci afin d'éliminer un maximum de NaN tout en gardant une cohérence dans les données.

In [37]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16538 non-null  object        
 5   Studio                16538 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

Notre dataset est presque complet.<br>
On élimine les colonnes devenues inutiles

In [38]:
df_rm=df_rm.drop(["GK_distributeur","Rate","Score","Review"],axis=1)

Les NaN restant seront gérés pendant le pré-processing, au moment de la modélisation.

In [39]:
df_rm.to_csv('../data/03_GameSpy_RollingMeans.csv')